In [2]:
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker
from torch.nn.parameter import Parameter
import matplotlib as mpl

import numpy as np
import time
from pyDOE import lhs         #Latin Hypercube Sampling
import scipy.io
from scipy.io import savemat

from smt.sampling_methods import LHS

#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())


cuda:2


In [ ]:
# def true_2D_1(xt): #True function for 2D_1 Klein Gordon Equation x \in [-50,50] , t \in [0,10]
#     y = xt[:,0]*np.cos(xt[:,1])
#     return y.reshape(-1,1)    

In [3]:
#Material Properties This link - https://www.mathworks.com/help/pde/ug/nonlinear-heat-transfer-in-a-thin-plate.html#heatTransferThinPlateExample-1
k = 400
rho = 8960
cp = 386
t_z = 0.01
stef_bolt = 5.670373e-8
hc = 1
Ta = 300
emiss = 0.5


In [4]:
label = "Heat_tanh"

x = np.linspace(0,1,100).reshape(-1,1)
y = np.linspace(0,1,100).reshape(-1,1)
t = np.linspace(0,1,100).reshape(-1,1) #t is actually from 0 to 5000, let us scale it to 0 to 1

X,Y,T = np.meshgrid(x,y,t)

X = X.flatten('F').reshape(-1,1)
Y = Y.flatten('F').reshape(-1,1)
T = T.flatten('F').reshape(-1,1)
  
xyt = np.hstack((X,Y,T))

initial_pts = np.logical_and(T==0,Y!=0).reshape(-1,)

DBC_pts = (Y == 0).reshape(-1,)


NBC_pts_x0 = (X == 0).reshape(-1,)
NBC_pts_x1 = (X == 1).reshape(-1,)

NBC_pts_y0 = (Y == 0).reshape(-1,)
NBC_pts_y1 = (Y == 1).reshape(-1,)

xyt_initial = xyt[initial_pts,:]
xyt_DBC = xyt[DBC_pts,:]

xyt_NBC_x0 = xyt[NBC_pts_x0,:]
xyt_NBC_x1 = xyt[NBC_pts_x1,:]

#xyt_NBC_y0 = xyt[NBC_pts_y0,:]
xyt_NBC_y1 = xyt[NBC_pts_y1,:]

u_initial = 300*np.ones((np.shape(xyt_initial)[0],1))
u_DBC = 1000*np.ones((np.shape(xyt_DBC)[0],1))

xyt_I_DBC = np.vstack((xyt_initial,xyt_DBC))
#xyt_NBC = np.vstack((xyt_NBC_1,xyt_NBC_2,xyt_NBC_3,xyt_NBC_4))
xyt_NBC_x = np.vstack((xyt_NBC_x0,xyt_NBC_x1))
#xyt_NBC_y = np.vstack((xyt_NBC_y0,xyt_NBC_y1))
xyt_NBC_y = np.vstack((xyt_NBC_y1))

u_I_DBC = np.vstack((u_initial,u_DBC))

xyt_test_tensor = torch.from_numpy(xyt).float().to(device)

lb_xyt = xyt[0]
ub_xyt = xyt[-1]

In [5]:
def trainingdata(N_D,N_N,N_f,seed):
    '''Boundary Conditions''' 
    
    np.random.seed(seed)
    
    #choose random N_u points for training
    idx = np.random.choice(xyt_I_DBC.shape[0], N_D, replace=False) 
    xyt_D = xyt_I_DBC[idx,:] #choose indices from  set 'idx' (x,t)
    u_D = u_I_DBC[idx].reshape(-1,1)      #choose corresponding u

    idx = np.random.choice(xyt_NBC_x.shape[0], N_D, replace=False) 
    xyt_Nx = xyt_NBC_x[idx,:] #choose indices from  set 'idx' (x,t)

    idx = np.random.choice(xyt_NBC_y.shape[0], N_D, replace=False) 
    xyt_Ny = xyt_NBC_y[idx,:] #choose indices from  set 'idx' (x,t)

    '''Collocation Points'''
    # Latin Hypercube sampling for collocation points 
    # N_f sets of tuples(x,t)
    x01 = np.array([[0.0,1.0],[0.0,1.0],[0.0,1.0]])
    sampling = LHS(xlimits=x01,random_state =seed)
    samples = sampling(N_f)
    
    xyt_coll = lb_xyt + (ub_xyt - lb_xyt)*samples
    xyt_coll = np.vstack((xyt_coll, xyt_D,xyt_Nx,xyt_Ny)) # append training points to collocation points 

    return xyt_coll, xyt_D, u_D, xyt_Nx,xyt_Ny

In [8]:
class Sequentialmodel(nn.Module):
    
    def __init__(self,layers):
        super().__init__() #call __init__ from parent class 
              
        'activation function'
        self.activation = nn.Tanh()

     
        'loss function'
        self.loss_function = nn.MSELoss(reduction ='mean')
        
        'Initialise neural network as a list using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        self.iter = 0
        
        # std = gain * sqrt(2/(input_dim+output_dim))
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data)   
        
#         self.beta = Parameter(torch.ones((50,len(layers)-2)))
#         self.beta.requiresGrad = True
        
        self.alpha = Parameter(torch.zeros(5,len(layers)-2))
        self.omega = Parameter(torch.ones(5,len(layers)-2))
        
    
        self.train_loss = []
        self.test_loss = []
        
        self.beta_val = []
        
        self.n = torch.tensor(1.0)
            
    'foward pass'
    def forward(self,xyt):
        if torch.is_tensor(xyt) != True:         
            xyt = torch.from_numpy(xyt)                
        
        ubxyt = torch.from_numpy(ub_xyt).float().to(device)
        lbxyt = torch.from_numpy(lb_xyt).float().to(device)
    
                      
        #preprocessing input 
        xyt = (xyt - lbxyt)/(ubxyt - lbxyt)
        
        #convert to float
        a = xyt.float()
        
        for i in range(len(layers)-2):
            z = self.linears[i](a)
            a = self.activation(z)
            #print(z.shape)
            #z2 = torch.zeros(z1.shape).to(device)
            for j in range(5):
                #print(j)
                a = a + self.alpha[j,i]*self.n*torch.sin((j+1)*self.n*self.omega[j,i]*z)
                
        a = self.linears[-1](a) 
         
        return a
                        
    def loss_D(self,xyt_D,u_D):
                
        loss_bc = self.loss_function(self.forward(xyt_D), u_D)
                
        return loss_bc
    
    def loss_N(self,xyt_Nx,xyt_Ny,N_hat):
        
        g1 = xyt_Nx.clone()             
        g1.requires_grad = True
        u1 = self.forward(g1)
        
        u1_x_y_t = autograd.grad(u1,g1,torch.ones([xyt_Nx.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        du1_dx = u1_x_y_t[:,[0]]
        
        g2 = xyt_Ny.clone()             
        g2.requires_grad = True
        u2 = self.forward(g2)
        
        u2_x_y_t = autograd.grad(u2,g2,torch.ones([xyt_Ny.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        du2_dy = u2_x_y_t[:,[1]]
               
        loss_N1 = self.loss_function(du1_dx,N_hat)
        loss_N2 = self.loss_function(du2_dy,N_hat)
        
        #return loss_N1+loss_N2       
        return loss_N1 + loss_N2
    
    def loss_PDE(self, xyt_coll, f_hat):
        
        g = xyt_coll.clone()             
        g.requires_grad = True
        u = self.forward(g) 
        
        u_x_y_t = autograd.grad(u,g,torch.ones([xyt_coll.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        u_xx_yy_tt = autograd.grad(u_x_y_t,g,torch.ones(xyt_coll.shape).to(device), create_graph=True,allow_unused = True)[0]

        du_dt = u_x_y_t[:,[2]]
        
        d2u_dx2 = u_xx_yy_tt[:,[0]]
        d2u_dy2 = u_xx_yy_tt[:,[1]]    
        

        f = rho*cp*t_z*du_dt/3000 - k*t_z*(d2u_dx2+d2u_dy2) + 2*hc*(u-Ta) + 2*emiss*stef_bolt*(torch.pow(u,4)-Ta**4) 
        
        loss_f = self.loss_function(f,f_hat)
                
        return loss_f
    
    def loss(self,xyt_D,u_D,xyt_Nx,xyt_Ny,N_hat,xyt_coll,f_hat):

        loss_D = self.loss_D(xyt_D,u_D)
        loss_N = self.loss_N(xyt_Nx,xyt_Ny,N_hat)
        loss_f = self.loss_PDE(xyt_coll,f_hat)
        
        loss_val = loss_D + loss_N + loss_f
        
        #print(self.iter,"loss_D:",loss_D.cpu().detach().numpy(),"loss_N:",loss_N.cpu().detach().numpy(),"loss_f:",loss_f.cpu().detach().numpy())
        
        return loss_val
     
    'callable for optimizer'                                       
    def closure(self):
        optimizer.zero_grad()
        
        loss = self.loss(xyt_D,u_D,xyt_Nx,xyt_Ny,N_hat,xyt_coll,f_hat)
        self.train_loss.append(loss.cpu().detach().numpy())
        
        #u_pred = self.test(xyt_test_tensor)
        #self.test_loss.append(np.mean(np.square(u_pred.reshape(-1,1) - u_true.reshape(-1,1)))) #Commented because no true values yet
        #self.beta_val.append(self.alpha.cpu().detach().numpy())
        
        #print(self.iter,"Train Loss",self.train_loss[-1],"Test Loss",self.test_loss[-1])
        print(self.iter,"Train Loss",self.train_loss[-1])
        
        loss.backward()
        self.iter += 1
  

        return loss        
    'test neural network'
    def test(self,xyt_test_tensor):
        u_pred = self.forward(xyt_test_tensor)
        u_pred = u_pred.cpu().detach().numpy()
   
        return u_pred

In [9]:
max_reps = 1

train_loss_full = []
test_loss_full = []
#alpha_full = []
elapsed_time= np.zeros((max_reps,1))

#for reps in range(max_reps):
for reps in range(8,9):
    print(reps)
    
    torch.manual_seed(reps*36)
    N_D = 5000 #Total number of data points for 'y'
    N_N = 3500
    N_f = 10000#Total number of collocation points 
    
    xyt_coll_np_array, xyt_D_np_array, u_D_np_array,xyt_Nx_np_array,xyt_Ny_np_array = trainingdata(N_D,N_N,N_f,(reps)*22)
        
    xyt_coll = torch.from_numpy(xyt_coll_np_array).float().to(device)
    xyt_D = torch.from_numpy(xyt_D_np_array).float().to(device)
    u_D = torch.from_numpy(u_D_np_array).float().to(device)
    xyt_Nx = torch.from_numpy(xyt_Nx_np_array).float().to(device)
    xyt_Ny = torch.from_numpy(xyt_Ny_np_array).float().to(device)
        
    N_hat = torch.zeros(xyt_Nx.shape[0],1).to(device)    
    f_hat = torch.zeros(xyt_coll.shape[0],1).to(device)

    layers = np.array([3,50,50,50,50,50,50,50,50,50,1]) #9 hidden layers
    #layers = np.array([3,100,100,100,100,100,100,100,1])

    PINN = Sequentialmodel(layers)
   
    PINN.to(device)

    'Neural Network Summary'
    print(PINN)

    params = list(PINN.parameters())
    
    optimizer = torch.optim.LBFGS(PINN.parameters(), lr=1, 
                              max_iter = 10000, 
                              max_eval = None, 
                              tolerance_grad = -1, 
                              tolerance_change = -1, 
                              history_size = 100, 
                              line_search_fn = 'strong_wolfe')



    start_time = time.time()
        
    optimizer.step(PINN.closure)
    
    torch.save(PINN.state_dict(),label+'_'+str(reps)+'.pt')
    train_loss_full.append(PINN.train_loss)
    test_loss_full.append(PINN.test_loss)
    elapsed_time[reps] = time.time() - start_time
    #alpha_full.append(PINN.alpha_val)


    print('Training time: %.2f' % (elapsed_time[reps]))
    
mdic = {"train_loss": train_loss_full,"test_loss": test_loss_full,"Time": elapsed_time,  "label": label}
savemat(label+'.mat', mdic)

8
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=3, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=1, bias=True)
  )
)
0 Train Loss 1683767.5
1 Train Loss 1679334.8
2 Train Loss 1668579.2
3 Train Loss 1668396.8
4 Train Loss 1667677.5
5 Train Loss 1666105.8
6 Train Loss 1654859.6
7 Train Loss 1738259.2
8 Train Loss 1636305.0
9 Train Loss 167865200.0
10 Train Loss 2169734.0
11 Train Loss 1643198.0
12 Train Loss 162834

306 Train Loss 106392.55
307 Train Loss 97675.67
308 Train Loss 96181.35
309 Train Loss 94782.64
310 Train Loss 95608.516
311 Train Loss 94361.59
312 Train Loss 93794.45
313 Train Loss 93253.336
314 Train Loss 92707.16
315 Train Loss 92225.56
316 Train Loss 92112.16
317 Train Loss 91956.984
318 Train Loss 91693.234
319 Train Loss 91593.01
320 Train Loss 91535.08
321 Train Loss 91433.66
322 Train Loss 91353.664
323 Train Loss 91261.55
324 Train Loss 91131.19
325 Train Loss 90988.05
326 Train Loss 90784.66
327 Train Loss 90571.766
328 Train Loss 90312.7
329 Train Loss 89757.29
330 Train Loss 89180.82
331 Train Loss 88860.19
332 Train Loss 88569.414
333 Train Loss 88752.71
334 Train Loss 88511.68
335 Train Loss 88392.195
336 Train Loss 88278.77
337 Train Loss 88129.89
338 Train Loss 87884.17
339 Train Loss 87781.59
340 Train Loss 87550.86
341 Train Loss 86751.3
342 Train Loss 87720.84
343 Train Loss 85924.64
344 Train Loss 87672.77
345 Train Loss 85238.23
346 Train Loss 84846.82
347 Train

643 Train Loss 48702.688
644 Train Loss 48686.234
645 Train Loss 48656.09
646 Train Loss 48601.688
647 Train Loss 48537.105
648 Train Loss 48506.33
649 Train Loss 48480.855
650 Train Loss 48432.363
651 Train Loss 48394.92
652 Train Loss 48330.43
653 Train Loss 48287.523
654 Train Loss 48239.664
655 Train Loss 48224.527
656 Train Loss 48189.246
657 Train Loss 48150.508
658 Train Loss 48103.445
659 Train Loss 48057.03
660 Train Loss 48022.875
661 Train Loss 47999.477
662 Train Loss 47942.75
663 Train Loss 47902.883
664 Train Loss 47886.324
665 Train Loss 47862.37
666 Train Loss 47850.95
667 Train Loss 47825.195
668 Train Loss 47764.332
669 Train Loss 47708.55
670 Train Loss 47650.195
671 Train Loss 47580.48
672 Train Loss 47733.14
673 Train Loss 47551.23
674 Train Loss 47444.152
675 Train Loss 47390.39
676 Train Loss 47319.035
677 Train Loss 47306.38
678 Train Loss 47245.266
679 Train Loss 47174.777
680 Train Loss 47021.445
681 Train Loss 47040.08
682 Train Loss 46937.332
683 Train Loss 

977 Train Loss 37847.203
978 Train Loss 37868.145
979 Train Loss 37831.535
980 Train Loss 37802.992
981 Train Loss 37834.938
982 Train Loss 37779.36
983 Train Loss 37763.516
984 Train Loss 37763.145
985 Train Loss 37743.402
986 Train Loss 37732.207
987 Train Loss 37742.742
988 Train Loss 37726.387
989 Train Loss 37719.54
990 Train Loss 37711.758
991 Train Loss 37703.547
992 Train Loss 37694.31
993 Train Loss 37688.06
994 Train Loss 37689.094
995 Train Loss 37686.824
996 Train Loss 37685.574
997 Train Loss 37684.367
998 Train Loss 37679.97
999 Train Loss 37677.156
1000 Train Loss 37672.87
1001 Train Loss 37666.76
1002 Train Loss 37657.645
1003 Train Loss 37653.973
1004 Train Loss 37645.625
1005 Train Loss 37640.71
1006 Train Loss 37638.066
1007 Train Loss 37635.656
1008 Train Loss 37634.344
1009 Train Loss 37632.297
1010 Train Loss 37626.465
1011 Train Loss 37620.094
1012 Train Loss 37614.97
1013 Train Loss 37616.082
1014 Train Loss 37611.598
1015 Train Loss 37609.04
1016 Train Loss 376

1299 Train Loss 35973.297
1300 Train Loss 35971.004
1301 Train Loss 35972.414
1302 Train Loss 35968.566
1303 Train Loss 35966.08
1304 Train Loss 35963.945
1305 Train Loss 35962.55
1306 Train Loss 35961.08
1307 Train Loss 35959.66
1308 Train Loss 35959.24
1309 Train Loss 35958.77
1310 Train Loss 35958.09
1311 Train Loss 35957.2
1312 Train Loss 35955.43
1313 Train Loss 35954.008
1314 Train Loss 35951.69
1315 Train Loss 35950.477
1316 Train Loss 35946.137
1317 Train Loss 35950.332
1318 Train Loss 35943.27
1319 Train Loss 35936.82
1320 Train Loss 35927.074
1321 Train Loss 35933.383
1322 Train Loss 35916.8
1323 Train Loss 36375.445
1324 Train Loss 35904.73
1325 Train Loss 35880.18
1326 Train Loss 35863.094
1327 Train Loss 35989.72
1328 Train Loss 35849.79
1329 Train Loss 35874.195
1330 Train Loss 35837.066
1331 Train Loss 35828.426
1332 Train Loss 35820.348
1333 Train Loss 35807.72
1334 Train Loss 35798.96
1335 Train Loss 35783.484
1336 Train Loss 35749.953
1337 Train Loss 35777.914
1338 Tr

...

11424 Train Loss 458619.03
11425 Train Loss 24568.98
11426 Train Loss 21463.754
11427 Train Loss 21420.863
11428 Train Loss 21420.896
11429 Train Loss 21420.912
11430 Train Loss 21420.807
11431 Train Loss 21420.746
11432 Train Loss 458619.03
11433 Train Loss 24568.98
11434 Train Loss 21463.754
11435 Train Loss 21420.863
11436 Train Loss 21420.896
11437 Train Loss 21420.912
11438 Train Loss 21420.807
11439 Train Loss 21420.746
11440 Train Loss 458619.03
11441 Train Loss 24568.98
11442 Train Loss 21463.754
11443 Train Loss 21420.863
11444 Train Loss 21420.896
11445 Train Loss 21420.912
11446 Train Loss 21420.807
11447 Train Loss 21420.746
11448 Train Loss 458619.03
11449 Train Loss 24568.98
11450 Train Loss 21463.754
11451 Train Loss 21420.863
11452 Train Loss 21420.896
11453 Train Loss 21420.912
11454 Train Loss 21420.807
11455 Train Loss 21420.746
11456 Train Loss 458619.03
11457 Train Loss 24568.98
11458 Train Loss 21463.754
11459 Train Loss 21420.863
11460 Train Loss 21420.896
11461 

11730 Train Loss 21463.754
11731 Train Loss 21420.863
11732 Train Loss 21420.896
11733 Train Loss 21420.912
11734 Train Loss 21420.807
11735 Train Loss 21420.746
11736 Train Loss 458619.03
11737 Train Loss 24568.98
11738 Train Loss 21463.754
11739 Train Loss 21420.863
11740 Train Loss 21420.896
11741 Train Loss 21420.912
11742 Train Loss 21420.807
11743 Train Loss 21420.746
11744 Train Loss 458619.03
11745 Train Loss 24568.98
11746 Train Loss 21463.754
11747 Train Loss 21420.863
11748 Train Loss 21420.896
11749 Train Loss 21420.912
11750 Train Loss 21420.807
11751 Train Loss 21420.746
11752 Train Loss 458619.03
11753 Train Loss 24568.98
11754 Train Loss 21463.754
11755 Train Loss 21420.863
11756 Train Loss 21420.896
11757 Train Loss 21420.912
11758 Train Loss 21420.807
11759 Train Loss 21420.746
11760 Train Loss 458619.03
11761 Train Loss 24568.98
11762 Train Loss 21463.754
11763 Train Loss 21420.863
11764 Train Loss 21420.896
11765 Train Loss 21420.912
11766 Train Loss 21420.807
11767

12036 Train Loss 21420.896
12037 Train Loss 21420.912
12038 Train Loss 21420.807
12039 Train Loss 21420.746
12040 Train Loss 458619.03
12041 Train Loss 24568.98
12042 Train Loss 21463.754
12043 Train Loss 21420.863
12044 Train Loss 21420.896
12045 Train Loss 21420.912
12046 Train Loss 21420.807
12047 Train Loss 21420.746
12048 Train Loss 458619.03
12049 Train Loss 24568.98
12050 Train Loss 21463.754
12051 Train Loss 21420.863
12052 Train Loss 21420.896
12053 Train Loss 21420.912
12054 Train Loss 21420.807
12055 Train Loss 21420.746
12056 Train Loss 458619.03
12057 Train Loss 24568.98
12058 Train Loss 21463.754
12059 Train Loss 21420.863
12060 Train Loss 21420.896
12061 Train Loss 21420.912
12062 Train Loss 21420.807
12063 Train Loss 21420.746
12064 Train Loss 458619.03
12065 Train Loss 24568.98
12066 Train Loss 21463.754
12067 Train Loss 21420.863
12068 Train Loss 21420.896
12069 Train Loss 21420.912
12070 Train Loss 21420.807
12071 Train Loss 21420.746
12072 Train Loss 458619.03
12073

12342 Train Loss 21420.807
12343 Train Loss 21420.746
12344 Train Loss 458619.03
12345 Train Loss 24568.98
12346 Train Loss 21463.754
12347 Train Loss 21420.863
12348 Train Loss 21420.896
12349 Train Loss 21420.912
12350 Train Loss 21420.807
12351 Train Loss 21420.746
12352 Train Loss 458619.03
12353 Train Loss 24568.98
12354 Train Loss 21463.754
12355 Train Loss 21420.863
12356 Train Loss 21420.896
12357 Train Loss 21420.912
12358 Train Loss 21420.807
12359 Train Loss 21420.746
12360 Train Loss 458619.03
12361 Train Loss 24568.98
12362 Train Loss 21463.754
12363 Train Loss 21420.863
12364 Train Loss 21420.896
12365 Train Loss 21420.912
12366 Train Loss 21420.807
12367 Train Loss 21420.746
12368 Train Loss 458619.03
12369 Train Loss 24568.98
12370 Train Loss 21463.754
12371 Train Loss 21420.863
12372 Train Loss 21420.896
12373 Train Loss 21420.912
12374 Train Loss 21420.807
12375 Train Loss 21420.746
12376 Train Loss 458619.03
12377 Train Loss 24568.98
12378 Train Loss 21463.754
12379 

IndexError: index 8 is out of bounds for axis 0 with size 1

In [40]:
del xyt_coll
del xyt_D

NameError: name 'xyt_coll' is not defined

In [42]:
del xyt_Nx
del xyt_Ny

In [36]:
torch.save(PINN.state_dict(),label+'_'+str(reps)+'.pt')

In [41]:
del xyt_test_tensor


NameError: name 'xyt_test_tensor' is not defined

In [43]:
xyt_test_tensor = torch.from_numpy(xyt[1:50000,:]).float().to(device)

In [44]:
u_pred1 = PINN.test(xyt_test_tensor)
#u_pred2 = PINN.test(xyt_test_tensor[200000:,:])

#u_pred = np.vstack((u_pred1,u_pred2))

u_pred_3d = u_pred.reshape(100,100,100,order = 'F')

RuntimeError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 2; 10.76 GiB total capacity; 9.63 GiB already allocated; 3.44 MiB free; 9.63 GiB reserved in total by PyTorch)

In [ ]:
fig, ax = plt.subplots(1,1)
cmap = plt.cm.jet

img3 = ax.imshow(np.flipud(u_pred_3d[:,:,50]),vmax =1000,vmin=450,cmap = cmap,extent=[0,1,0,1],aspect = 0.75)
fig.colorbar(img3, orientation='vertical')

In [ ]:
import subprocess
import os

In [ ]:
def generate_video():
    for i in range(100):
        #plt.imshow(img[i], cmap=cm.Greys_r)
        img3 = ax.imshow(np.flipud(u_pred_3d[:,:,i]),vmax =1000,vmin=450,cmap = cmap,extent=[0,1,0,1],aspect = 0.75)
        
        plt.savefig("stan_heat_%02d.png" % i)

    #os.chdir()
    subprocess.call(['ffmpeg', '-framerate', '8', '-i', 'stan_heat_%02d.png', '-r', '10', '-pix_fmt', 'yuv420p','Stan_transient.mp4'])
    for file_name in glob.glob("*.png"):
        os.remove(file_name)

In [ ]:
generate_video()

In [ ]:
import matplotlib.animation as animation

fig, ax = plt.subplots(1,1)
# img = [] # some array of images
frames = [] # for storing the generated images
# fig = plt.figure()
for i in range(100):
#     p1 = ax.imshow(np.flipud(u_pred_3d[:,:,i]),vmax =1000,vmin=450,cmap = cmap,extent=[0,1,0,1],aspect = 0.75)
#     fig.colorbar(p1, orientation='vertical',ax=ax)
    frames.append([plt.imshow(np.flipud(u_pred_3d[:,:,i]),vmax =1000,vmin=450,cmap = cmap,extent=[0,1,0,1],aspect = 0.75)])
    print(i)

ani = animation.ArtistAnimation(fig, frames, interval=50, blit=True,
                                repeat_delay=1000)
#ani.save('movie.mp4')
plt.show()

In [ ]:
writergif = animation.PillowWriter(fps=10)
ani.save('rowdy_Thinplate_movie.gif',writer=writergif)

In [ ]:
writervideo = animation.FFMpegWriter(fps=60) 
ani.save('Stan_Thinplate_movie1.gif',writer=writervideo)

In [ ]:
np.max(u_pred_3d[:,:,50])

In [ ]:
plt.plot(u_pred_3d[50,99,:])

In [ ]:
a = 0
for i in range(10):
    a = a + test_loss_full[i][-1]
print(a/10)

In [ ]:
a = 0
for i in range(10):
    a = a + train_loss_full[i][-1]
print(a/10)